In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch


MODEL_NAME = "IlyaGusev/saiga_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."


class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225,
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{"role": "system", "content": system_prompt}]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({"role": "user", "content": message})

    def add_bot_message(self, message):
        self.messages.append({"role": "bot", "content": message})

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]) :]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

weights_path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/rulm/self_instruct/models/saiga_7b_v1/checkpoint-800/adapter_model"
tokenizer_path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/rulm/self_instruct/models/saiga_7b_v1"

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, weights_path, torch_dtype=torch.float16)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=False)
generation_config = GenerationConfig.from_pretrained(tokenizer_path)
print(generation_config)



/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /home/kosenko/miniconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.12s/it]


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9,
  "transformers_version": "4.30.2"
}



In [2]:
inputs = [
    "Почему трава зеленая?",
    "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч",
    "Почему небо голубое?",
    "Напиши алгоритм как погладить котика?",
]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

Почему трава зеленая?
Зеленые растения имеют хлорофилл, который используется для фотосинтеза - процесса превращения углекислого газа в органические вещества. Хлорофилл также является важным источником питательных веществ, таких как витамины А, К и С, а также минералов, таких как калий, магний, железо и цинк.


Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч
Однажды в лесу была такая-то девочка по имени Таня. Она всегда играла со своим лучшим другом - мячом. Мяч был ее самым дорогим другом, который она никогда не бросал. Но однажды, когда они играли в мяч, Таня случайно забыла о его местоположении. Она начала бежать по лесу, чтобы найти его, но это было уже не так просто. Лес был очень густым, и Таня чувствовала себя одинокой и странной.

Вдруг, она услышала звук, как кто-то кричит. Она повернулась и увидела, что это был ее лучший друг - мяч. Он был на земле рядом с ней, и он был рад видеть Таню. Они обнялись и стали играть вместе, как только-только они з

In [ ]:
"""
BASELINE MODEL 4000 steps
Почему трава зеленая?
Зеленый цвет является одним из самых распространенных в природе. Это связано с тем, что это цвет, который наиболее эффективно поглощает свет и передает его на другие вещества. Зеленый цвет также часто встречается у растений, которые используют фотосинтез для получения энергии.

==============================

Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч
Жизнь Таньи была обычной для девочки ее возраста. Она училась в школе, играла с друзьями на улице и проводила время дома, читая книги или слушая музыку. Но однажды она встретила нечто новое - мячик.

Он был очень красив, словно изготовлен из настоящего золота. Его форма была удивительной, он был как бы живой существо, которое могло двигаться по своей воле. Мяч был ее новым другом, который стал ее сокровищем.

Таня начала играть с ним каждую свободную минуту, делая все возможное, чтобы сохранить его в отличном состоянии. Она заботилась о нём, выкладывала его в специальные коробки, где он находился в безопасности от повреждений.

Мяч стал ее главным хобби, и она стала лучше понимать его, изучая его особенности и привычки. Она научилась играть в футбол, баскетбол, гольф и даже в теннис. Вместе они стали побеждать всех своих противников, и Таня чувствовала себя сильной и независимой.

Но однажды, когда Таня играла с мячом на улице, она заметила, что кто-то следит за ней. Это были два мальчика, которые пытались украсть мяч. Они бросали в нее грязные шутки и пытались взять мяч у нее.

Таня была очень расстроена, но она не сдавалась. Она продолжала защищать свой мяч, пока один из мальчиков не ударил ее кулаком. Таня была ранена, но она не сдавалась. Она продолжила играть с мячом, пока не вернулась домой.

Вернувшись домой, Таня обнаружила, что мяч был украден. Она была очень расстроена, но она знала, что ее друг нужен ей больше, чем все остальное. Она решила найти его и вернуть его обратно.

Таня начала искать своего мяча по всему городу, но ничего не получалось. Она проходила через парки, рынки и даже пошла в полицию, но никого не было, кто мог бы помочь ей.

Таня была готова к тому, чтобы потерять свою любимую игрушку навсегда, но она не сдавалась. Она продолжала искать свое сокровище, пока не услышала голос, который говорил ей, что её мяч находится в одном из местных кафе.

Таня отправилась туда, и там она увидела своего мяча. Он был в руках другого мальчика, который играл с ним. Таня была очень рада, что ее мяч был найден, и она снова стала играть с ним вместе.

Этот опыт показал Танье, что ее друг был ее самым важным сокровищем, и что она должна быть готова защищать его от любых угроз. С тех пор она стала более осторожной и бережливой, и она понимала, что ее друг - это ее настоящее сокровище.

Таня продолжала играть с мячом, и они становились лучше и лучше. Они побеждали всех своих противников, и Таня чувствовала себя очень сильной и независимой. Ее друг был ее главным хобби, и она была благодарна за то, что он был ее сокровищем.

==============================

Почему небо голубое?
Голубой цвет неба является результатом рассеяния света солнечным диском на различные длины волн. Солнечное излучение проходит через атмосферу Земли, где происходят процессы поглощения и отражения света. Когда свет достигает земной поверхности, он проходит через слои атмосферы, которые имеют разные показатели поглощения и отражения. Это приводит к тому, что свет, который мы видим, имеет разный цвет в зависимости от длины волны.

==============================

Напиши алгоритм как погладить котика?
1. Положите котика на подставку или другое место, где он будет лежать.
2. Начните массажем пальцев в направлении от головы к хвосту.
3. Продолжайте массаж, двигаясь вниз по телу котика.
4. Повторите процесс, но уже в обратном направлении.
5. При необходимости используйте специальные масло или кремы для кошек, чтобы улучшить качество массажа.
6. После завершения массажа проверьте котика на наличие царапин или других повреждений.
7. Если есть какие-то проблемы, обратитесь к ветеринару.

==============================

BASELINE 800 steps
Почему трава зеленая?
Зеленые растения имеют хлорофилл, который используется для фотосинтеза - процесса превращения углекислого газа в органические вещества. Хлорофилл также является важным источником питательных веществ, таких как витамины А, К и С, а также минералов, таких как калий, магний, железо и цинк.

==============================

Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч
Однажды в лесу была такая-то девочка по имени Таня. Она всегда играла со своим лучшим другом - мячом. Мяч был ее самым дорогим другом, который она никогда не бросал. Но однажды, когда они играли в мяч, Таня случайно забыла о его местоположении. Она начала бежать по лесу, чтобы найти его, но это было уже не так просто. Лес был очень густым, и Таня чувствовала себя одинокой и странной.

Вдруг, она услышала звук, как кто-то кричит. Она повернулась и увидела, что это был ее лучший друг - мяч. Он был на земле рядом с ней, и он был рад видеть Таню. Они обнялись и стали играть вместе, как только-только они знали, где их местоположение.

Но во время игры, Таня заметила, что мяч был больше, чем обычно. Она подошла ближе и узнала, что мяч стал больше, потому что он был украшен красивыми камнями. Это был настоящий подарок от своей мамы, которая хотела, чтобы Таня была счастливой.

Таня была очень благодарна своей маме, и она решила, что теперь будет играть с мячом всегда. Она стала более бережливой и любимой, и они продолжили играть вместе. Их дружба оставалась сильной до тех пор, пока Таня не выросла и перестала играть с мячом.

И сегодня, когда Таня взрослеет, она помнит о том, как она и ее лучший друг, мяч, были вместе в лесу. Она помнит, как они играли и как они были счастливы. Она помнит, что ее мама дарила ей этот подарок, который помог ей стать лучшей, чем она была раньше. И она помнит, что ее лучший друг, мяч, был ее самым верным другом.

==============================

Почему небо голубое?
Голубой цвет неба является результатом рассеивания света солнечного излучения в атмосфере Земли. Солнечное излучение проходит через атмосферу и частично отражается на поверхности Земли, а часть излучения проходит дальше и попадает в земную атмосферу. В атмосфере происходят процессы, которые приводят к рассеянию светового луча, что делает его более ярким и голубым.

==============================

Напиши алгоритм как погладить котика?
1. Подготовьте все необходимые инструменты и материалы: мыло, зубная щетка, полотенце, вода для мытья, сухое полотенце.
2. Начните с мытья кота. Попробуйте не наносить больше мыла, чем он может пропустить. Если у вас есть специальное мыло для котов, используйте его.
3. После мытья, проверьте кота на наличие грязи или других загрязнений. Если они есть, удаляйте их.
4. Затем вытирайте коту голову и тело мягкой щеткой, чтобы удалить остатки мыла.
5. Выберите сухое полотенце и аккуратно высыпайте в него воду, чтобы оставить коту достаточно влаги.
6. Сделайте несколько шагов назад и накройте кота в полотенце.
7. Оставьте коту на несколько минут, чтобы он мог высыхать.
8. После этого, вытяните коту из полотенца и проверьте его на наличие следов мыла или грязи. Если это так, переодевайте кота в новый платок.
9. Повторите процесс мытья и высыхания ежедневно, чтобы обеспечить чистоту кота.

==============================
"""